In [29]:
import pandas as pd
import numpy as np
import pickle
import scanpy as sc
import sys
sys.path.append('../GENIE3/GENIE3_python/')
from GENIE3 import GENIE3
from matplotlib import pyplot as plt

In [47]:
# Supplementary Data
with open('../Data/SCP2797/file_supplemental_info.tsv', 'rb') as f:
    sup_data = pd.read_csv(f, delimiter='\t')

sup_data

,filename,file_type,species_scientific_name,genome_assembly_name,genome_assembly_accession,genome_annotation_name,is_raw_counts,library_preparation_protocol,units,biosample_input_type,modality
0,rs1.rs2.combined.counts.tsv.gz,Expression Matrix,Mus musculus,NaN,NaN,NaN,True,10x 3' v3,UMI-corrected raw counts,Single nuclei,Transcriptomic: unbiased
1,rs1.rs2.combined.data.tsv.gz,Expression Matrix,Mus musculus,NaN,NaN,NaN,NaN,10x 3' v3,NaN,Single nuclei,Transcriptomic: unbiased
2,metadata_AllRabies.tsv,Metadata,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rs1.rs2.combined.umap.scp.tsv,Cluster,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rs1.rs2.combined.rds,Seurat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Meta Data
with open('../Data/SCP2797/metadata/metadata_AllRabies.tsv', 'rb') as f:
    meta_data = pd.read_table(f)

meta_data

,NAME,biosample_id,donor_id,species,species__ontology_label,sex,cell_type,cell_type__ontology_label,disease,disease__ontology_label,organ,organ__ontology_label,library_preparation_protocol,library_preparation_protocol__ontology_label
0,TYPE,group,group,group,group,group,group,group,group,group,group,group,group,group
1,RABIES.B1_TTGACCCAGACCATTC-1,19,batch1,NCBITaxon_10090,Mus musculus,unknown,CL_0000540,neuron,PATO:0000461,normal,UBERON:0001898,hypothalamus,EFO:0009922,10x 3' v3
2,RABIES.B1_AATGGCTAGTGCAAAT-1,17,batch1,NCBITaxon_10090,Mus musculus,unknown,CL_0000540,neuron,PATO:0000461,normal,UBERON:0001898,hypothalamus,EFO:0009922,10x 3' v3
3,RABIES.B1_AACAAGAGTGCACAAG-1,13,batch1,NCBITaxon_10090,Mus musculus,unknown,CL_0000540,neuron,PATO:0000461,normal,UBERON:0001898,hypothalamus,EFO:0009922,10x 3' v3
4,RABIES.B1_CCCTCAATCAGCCTCT-1,17,batch1,NCBITaxon_10090,Mus musculus,unknown,CL_0000540,neuron,PATO:0000461,normal,UBERON:0001898,hypothalamus,EFO:0009922,10x 3' v3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5766,B2.PVH.PID_HTO.RF_TTTGGAGCATGGCTAT,24,batch2_refed,NCBITaxon_10090,Mus musculus,unknown,CL_0000540,neuron,PATO:0000461,normal,UBERON:0001898,hypothalamus,EFO:0009922,10x 3' v3
5767,B2.PVH.PID_HTO.RF_AGGTCATTCTCTGCCA,24,batch2_refed,NCBITaxon_10090,Mus musculus,unknown,CL_0000540,neuron,PATO:0000461,normal,UBERON:0001898,hypothalamus,EFO:0009922,10x 3' v3
5768,B2.PVH.PID_HTO.RF_GGGTGAAGTGATATAG,1,batch2_refed,NCBITaxon_10090,Mus musculus,unknown,CL_0000540,neuron,PATO:0000461,normal,UBERON:0001898,hypothalamus,EFO:0009922,10x 3' v3
5769,B2.PVH.PID_HTO.RF_TCAAGACCATCGAGCC,20,batch2_refed,NCBITaxon_10090,Mus musculus,unknown,CL_0000540,neuron,PATO:0000461,normal,UBERON:0001898,hypothalamus,EFO:0009922,10x 3' v3


In [36]:
meta_data = meta_data.drop(0).reset_index(drop=True)
meta_data

,NAME,biosample_id,donor_id,species,species__ontology_label,sex,cell_type,cell_type__ontology_label,disease,disease__ontology_label,organ,organ__ontology_label,library_preparation_protocol,library_preparation_protocol__ontology_label
0,RABIES.B1_TTGACCCAGACCATTC-1,19,batch1,NCBITaxon_10090,Mus musculus,unknown,CL_0000540,neuron,PATO:0000461,normal,UBERON:0001898,hypothalamus,EFO:0009922,10x 3' v3
1,RABIES.B1_AATGGCTAGTGCAAAT-1,17,batch1,NCBITaxon_10090,Mus musculus,unknown,CL_0000540,neuron,PATO:0000461,normal,UBERON:0001898,hypothalamus,EFO:0009922,10x 3' v3
2,RABIES.B1_AACAAGAGTGCACAAG-1,13,batch1,NCBITaxon_10090,Mus musculus,unknown,CL_0000540,neuron,PATO:0000461,normal,UBERON:0001898,hypothalamus,EFO:0009922,10x 3' v3
3,RABIES.B1_CCCTCAATCAGCCTCT-1,17,batch1,NCBITaxon_10090,Mus musculus,unknown,CL_0000540,neuron,PATO:0000461,normal,UBERON:0001898,hypothalamus,EFO:0009922,10x 3' v3
4,RABIES.B1_CGCATGGTCATTGTGG-1,17,batch1,NCBITaxon_10090,Mus musculus,unknown,CL_0000540,neuron,PATO:0000461,normal,UBERON:0001898,hypothalamus,EFO:0009922,10x 3' v3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5765,B2.PVH.PID_HTO.RF_TTTGGAGCATGGCTAT,24,batch2_refed,NCBITaxon_10090,Mus musculus,unknown,CL_0000540,neuron,PATO:0000461,normal,UBERON:0001898,hypothalamus,EFO:0009922,10x 3' v3
5766,B2.PVH.PID_HTO.RF_AGGTCATTCTCTGCCA,24,batch2_refed,NCBITaxon_10090,Mus musculus,unknown,CL_0000540,neuron,PATO:0000461,normal,UBERON:0001898,hypothalamus,EFO:0009922,10x 3' v3
5767,B2.PVH.PID_HTO.RF_GGGTGAAGTGATATAG,1,batch2_refed,NCBITaxon_10090,Mus musculus,unknown,CL_0000540,neuron,PATO:0000461,normal,UBERON:0001898,hypothalamus,EFO:0009922,10x 3' v3
5768,B2.PVH.PID_HTO.RF_TCAAGACCATCGAGCC,20,batch2_refed,NCBITaxon_10090,Mus musculus,unknown,CL_0000540,neuron,PATO:0000461,normal,UBERON:0001898,hypothalamus,EFO:0009922,10x 3' v3


In [2]:
with open('../Data/SCP2797/expression/rs1.rs2.combined.counts.tsv.gz', 'rb') as f:
    data = pd.read_table(f, compression='gzip')

data

,GENE,RABIES.B1_TTGACCCAGACCATTC-1,RABIES.B1_AATGGCTAGTGCAAAT-1,RABIES.B1_AACAAGAGTGCACAAG-1,RABIES.B1_CCCTCAATCAGCCTCT-1,RABIES.B1_CGCATGGTCATTGTGG-1,RABIES.B1_GAAGAATCAAACTCGT-1,RABIES.B1_ATGACCACATGTAACC-1,RABIES.B1_TCAATCTAGTATTCCG-1,RABIES.B1_TTCCTCTCACCAGACC-1,...,B2.PVH.PID_HTO.RF_AATCACGAGCGAGTCA,B2.PVH.PID_HTO.RF_TATCTGTGTACAGAAT,B2.PVH.PID_HTO.RF_CCCGGAATCTGCGGCA,B2.PVH.PID_HTO.RF_GATGCTAGTTAGCTAC,B2.PVH.PID_HTO.RF_CATCGCTTCGTCTCAC,B2.PVH.PID_HTO.RF_TTTGGAGCATGGCTAT,B2.PVH.PID_HTO.RF_AGGTCATTCTCTGCCA,B2.PVH.PID_HTO.RF_GGGTGAAGTGATATAG,B2.PVH.PID_HTO.RF_TCAAGACCATCGAGCC,B2.PVH.PID_HTO.RF_GGGTCTGTCCGGACGT
0,Xkr4,22,10,10,8,10,19,15,9,9,...,1,1,2,0,1,0,2,0,3,1
1,Gm1992,0,4,3,0,0,3,6,7,1,...,0,0,0,0,0,0,0,0,1,2
2,Gm19938,0,0,0,0,0,2,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,Gm37381,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Rp1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32280,AC124606.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32281,AC133095.2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32282,AC133095.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32283,AC234645.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
data = data.rename(columns={'GENE' : 'Sample'})
data = data.set_index('Sample')
data = data.transpose()
data

Sample,Xkr4,Gm1992,Gm19938,Gm37381,Rp1,Sox17,Gm37587,Gm37323,Mrpl15,Lypla1,...,Gm16367,AC163611.1,AC163611.2,AC140365.1,AC124606.2,AC124606.1,AC133095.2,AC133095.1,AC234645.1,AC149090.1
RABIES.B1_TTGACCCAGACCATTC-1,22,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
RABIES.B1_AATGGCTAGTGCAAAT-1,10,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,17
RABIES.B1_AACAAGAGTGCACAAG-1,10,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,17
RABIES.B1_CCCTCAATCAGCCTCT-1,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
RABIES.B1_CGCATGGTCATTGTGG-1,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B2.PVH.PID_HTO.RF_TTTGGAGCATGGCTAT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
B2.PVH.PID_HTO.RF_AGGTCATTCTCTGCCA,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
B2.PVH.PID_HTO.RF_GGGTGAAGTGATATAG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B2.PVH.PID_HTO.RF_TCAAGACCATCGAGCC,3,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# Get raw expression count matrix
exp_matrix = data.to_numpy()

with open('../Data/raw_exp_matrix.pickle', 'wb') as f:
    pickle.dump(exp_matrix, f)

In [10]:
exp_matrix = data.to_numpy()
exp_matrix.shape

(3005, 19968)

In [61]:
# Get highly variable genes 

# Creating AnnData Object from
adata = sc.AnnData(X=exp_matrix, obs=meta_data)

# with open('../Data/gene_exp_anndata.pickle', 'wb') as f:
#     pickle.dump(adata, f)

/home/ckotula/.local/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [62]:
# log-transforming data so we can find highly variable genes
sc.pp.log1p(adata)

In [72]:
# Find highly variable genes
sc.pp.highly_variable_genes(adata, n_top_genes=500)

In [78]:
# Getting top 500 most highly variable genes
highly_variable_idx = [i for i in range(adata.X.shape[1]) if adata.var['highly_variable'].iloc[i] == True]



503

In [86]:
highly_variable_genes = data.columns[highly_variable_idx].to_list()
highly_variable_genes

['St18',
 'Cpa6',
 'A830018L16Rik',
 'Sulf1',
 'Gm28376',
 'Gulp1',
 'Satb2',
 'Casp8',
 'Pard3b',
 'Pard3bos1',
 'Gm39653',
 'Pth2r',
 'Slc11a1',
 'Col4a4',
 'Col4a3',
 'Sphkap',
 'St8sia4',
 'Cntnap5b',
 'Cdh20',
 '9330185C12Rik',
 'Sctr',
 'Thsd7b',
 'Mgat4e',
 'Kif14',
 'Nr5a2',
 'Gm4258',
 'Gm26936',
 'Rgs13',
 'Gm15584',
 'Ptgs2',
 'Hmcn1',
 'Gm20631',
 'Pappa2',
 'Prrx1',
 'Lmx1a',
 'Gm10522',
 'F11r',
 'Ifi206',
 'Ifi213',
 'Ifi209',
 'Ifi203',
 'Pld5',
 'Gm37406',
 'Dnah14',
 'Atf3',
 'Kcnh1',
 'Plxdc2',
 'Etl4',
 'Fcnb',
 'Zeb2',
 'Gm13481',
 'Lypd6',
 'Neb',
 'Fap',
 'Fign',
 'Zfp385b',
 'Pde1a',
 'Zfp804a',
 'Fsip2',
 'Calcrl',
 'Olfr1056',
 'Prr5l',
 'Gm13912',
 'Bdnf',
 'Meis2',
 'Plcb2',
 'Terb2',
 'Hdc',
 'Mertk',
 'Oxt',
 'Avp',
 'Ovol2',
 'Cfap61',
 'Cst13',
 'Gm14139',
 'Vsx1',
 'Gm14170',
 'Bcas1',
 'Bcas1os2',
 'Gnasas1',
 'Crh',
 'Gm33206',
 'Kcnmb2',
 'Sox2ot',
 'Gm38505',
 'Trpc4',
 'C130079G13Rik',
 'Arhgef26',
 'Gpr149',
 'A330015K06Rik',
 'Veph1',
 'Gm20754',

In [87]:
with open('../Data/highly_variable_genes_503.pickle', 'wb') as f:
    pickle.dump(highly_variable_genes, f)

In [91]:
# Subsetting expression data to highly variable genes
highly_variable_genes_matrix = data[highly_variable_genes].to_numpy()

with open('../Data/highly_variable_expression_matrix_503.pickle', 'wb') as f:
    pickle.dump(highly_variable_genes_matrix, f)

In [43]:
# Run GENIE3
